In [182]:
import re
import os
import pandas as pd
import numpy as np
import nltk

In [183]:
def divide_into_chunks(text, chunk_size=3):
    # Tokenize the text into sentences
    sentences = nltk.sent_tokenize(text)

    # Divide the sentences into chunks
    chunks = [sentences[i:i + chunk_size] for i in range(0, len(sentences), chunk_size)]

    return chunks

In [184]:
file_names = [f for f in os.listdir("E:\\BSCS 4-3\\Thesises\\genective\\corpus")]

samples = []
for name in file_names:
    # open corpus file
    with open('./corpus/' + name, 'r', encoding="utf8") as file:
        corpus = file.read()
        
    # split into sentences
    document_chunks = divide_into_chunks(corpus)
    
    for chunk in document_chunks:
        sample = ' '.join(chunk)
        sample = ' '.join(filter(lambda s: 'http' not in s and 'www.' not in s, sample.split()))
        samples.append(sample)
    
        
samples = np.array(samples)
np.random.shuffle(samples)

samples[0]

'In the colon, inflammation also may be caused by a decrease in healthy bacteria and an increase in disease-causing bacteria. This change in the bacteria may permit chronic inflammation to develop in the colon. What is fiber?'

In [185]:
len(samples)

61395

In [186]:
gen_df = pd.DataFrame(samples, columns=['corpus'])
gen_df

,corpus
0,"In the colon, inflammation also may be caused ..."
1,The first number of the fraction is how many p...
2,"Much of this information comes from Orphanet, ..."
3,Signs and Symptoms Approximate number of patie...
4,Evaporative dry eye may result from inflammati...
...,...
61390,Your risk of infection depends on the number o...
61391,"SALL proteins act as transcription factors, wh..."
61392,The babys kidneys begin to produce urine at ab...
61393,Treatment Options for Hairy Cell Leukemia Untr...


In [187]:
gen_df.to_csv('./datasets/genetics.csv', index=False)

In [188]:
med_df = pd.read_csv('./corpus/medquad.csv')
med_df = med_df.drop(columns=['source', 'focus_area'])

med_df['corpus'] = med_df['question'] + ' ' + med_df['answer']
med_df = med_df.drop(columns=['question', 'answer'])

med_df['corpus'] = med_df['corpus'].astype(str)
med_df['corpus'] = med_df['corpus'].apply(lambda x: ' '.join(filter(lambda s: 'http' not in s and 'www.' not in s, x.split())))

med_df

,corpus
0,What is (are) Glaucoma ? Glaucoma is a group o...
1,What causes Glaucoma ? Nearly 2.7 million peop...
2,What are the symptoms of Glaucoma ? Symptoms o...
3,What are the treatments for Glaucoma ? Althoug...
4,What is (are) Glaucoma ? Glaucoma is a group o...
...,...
16407,What is (are) Diabetic Neuropathies: The Nerve...
16408,How to prevent Diabetic Neuropathies: The Nerv...
16409,How to diagnose Diabetic Neuropathies: The Ner...
16410,What are the treatments for Diabetic Neuropath...


In [189]:
med_df.to_csv('./datasets/general.csv', index=False)

In [190]:
df = pd.concat([gen_df, med_df])
df = df.sample(frac=1, random_state=64)
df

,corpus
15964,"A more precise measurement, such as a urine al..."
3244,What are the symptoms of Fraser like syndrome ...
16590,Most cases of shingles occur in adults. Only a...
7712,Is Bilateral perisylvian polymicrogyria inheri...
28794,""",GARD,Coccygodynia What are the treatments fo..."
...,...
23583,"In the fatal cases (4/5 patients), a transient..."
40755,This chemical robs your blood of oxygen and tr...
41969,Physical Activity Being physically active and ...
49206,""",GARD,Faciocardiorenal syndrome What is (are)..."


In [191]:
df.to_csv('./datasets/dataset.csv', index=False)